In [1]:
#导入需要的库
import numpy as np
import pandas as pd
import timeit

In [2]:
train_orig =  pd.read_csv('../data/train.csv', header=0)
test_orig =  pd.read_csv('../data/test.csv', header=0)

tic0=timeit.default_timer()
df1 = train_orig[['question1']].copy()
df2 = train_orig[['question2']].copy()
df1_test = test_orig[['question1']].copy()
df2_test = test_orig[['question2']].copy()

In [3]:
df2.rename(columns = {'question2':'question1'},inplace=True)
df2_test.rename(columns = {'question2':'question1'},inplace=True)

train_questions = df1.append(df2)
print(train_questions.shape)
train_questions = train_questions.append(df1_test)
print(train_questions.shape)
train_questions = train_questions.append(df2_test)
print(train_questions.shape)
#train_questions.drop_duplicates(subset = ['qid1'],inplace=True)
train_questions.drop_duplicates(subset = ['question1'],inplace=True)
print(train_questions.shape)

(808580, 1)
(3154376, 1)
(5500172, 1)
(4780684, 1)


In [4]:
train_questions.reset_index(inplace=True,drop=True)
questions_dict = pd.Series(train_questions.index.values,index=train_questions.question1.values).to_dict()

In [5]:
train_cp = train_orig.copy()
test_cp = test_orig.copy()
train_cp.drop(['qid1','qid2'],axis=1,inplace=True)

test_cp['is_duplicate'] = -1
test_cp.rename(columns={'test_id':'id'},inplace=True)
comb = pd.concat([train_cp,test_cp])

comb['q1_hash'] = comb['question1'].map(questions_dict)
comb['q2_hash'] = comb['question2'].map(questions_dict)

q1_vc = comb.q1_hash.value_counts().to_dict()#计算一个Series中各词出现的频率
q2_vc = comb.q2_hash.value_counts().to_dict()

In [6]:
comb[(comb.q1_hash==538457)]

,id,question1,question2,is_duplicate,q1_hash,q2_hash
2758,2758,did akbar ever war against extrovert son jahan...,if akbhar was it because of anarkali,-1,538457,2679105


In [9]:
def try_apply_dict(x,dict_to_apply):
    try:
        return dict_to_apply[x]
    except KeyError:
        return 0
comb['q1_freq'] = comb['q1_hash'].map(lambda x: try_apply_dict(x,q1_vc) + try_apply_dict(x,q2_vc))
comb['q2_freq'] = comb['q2_hash'].map(lambda x: try_apply_dict(x,q1_vc) + try_apply_dict(x,q2_vc))

train_comb = comb[comb['is_duplicate'] >= 0][['id','q1_hash','q2_hash','q1_freq','q2_freq','is_duplicate']]
test_comb = comb[comb['is_duplicate'] < 0][['id','q1_hash','q2_hash','q1_freq','q2_freq']]

In [10]:
print(train_comb.shape)
print(test_comb.shape)

(404290, 6)
(2345796, 5)


In [11]:
train_comb.to_csv('../train/train3.csv',index=None,encoding="utf-8")
test_comb.to_csv('../test/test3.csv',index=None,encoding="utf-8")
#comb.to_csv('../train/test3.csv',index=None,encoding="utf-8")

In [12]:
train_comb.head()

,id,q1_hash,q2_hash,q1_freq,q2_freq,is_duplicate
0,0,0,289987,1,2,0
1,1,1,16731,8,3,0
2,2,2,289988,2,1,0
3,3,3,289989,1,1,0
4,4,4,289990,3,1,0


In [13]:
corr_mat = train_comb.corr()#关于相关系数的计算可以调用pearson方法或kendell方法或spearman方法，默认使用pearson方法。
corr_mat.head(10)
#more frequenct questions are more likely to be duplicates

,id,q1_hash,q2_hash,q1_freq,q2_freq,is_duplicate
id,1.000000,0.691367,0.285360,-0.000673,-0.000632,-0.008784
q1_hash,0.691367,1.000000,0.491959,-0.285238,-0.189506,-0.207020
q2_hash,0.285360,0.491959,1.000000,-0.328662,-0.434609,-0.353183
q1_freq,-0.000673,-0.285238,-0.328662,1.000000,0.417006,0.243347
q2_freq,-0.000632,-0.189506,-0.434609,0.417006,1.000000,0.180834
is_duplicate,-0.008784,-0.207020,-0.353183,0.243347,0.180834,1.000000
